In [5]:
%%capture
!pip install numpy nltk pycryptodome
import numpy as np
import binascii
import base64
import nltk
import pprint
import json
nltk.download("words")
from nltk.corpus import words
from Crypto.Cipher import AES

# Table of Contents<a name="toc"></a>
* [Convert hex to base64](#prob1)
* [Fixed XOR](#prob2)
* [Single-byte XOR cipher](#prob3)
* [Detect single-character XOR](#prob4)
* [Implementing repeating-key XOR](#prob5)
* [Breaking repeating-key XOR](#prob6)
* [AES in ECB mode](#prob7)
* [Detect AES in ECB mode](#prob8)

# Convert hex to base64<a name="prob1"></a>

In [13]:
string_to_convert = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"

In [14]:
output = base64.b64encode(binascii.unhexlify(string_to_convert)).decode()
print(f"base64 encoded: {output}")

base64 encoded: SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t


# Fixed XOR<a name="prob2"></a>

In [16]:
cipher = "1c0111001f010100061a024b53535009181c"

In [17]:
xor_block = "686974207468652062756c6c277320657965"

In [2]:
def xor_encrypt(cipher: bytes, block: bytes) -> bytes:
    cipher_npa = np.frombuffer(cipher, dtype=np.uint8)
    block_npa = np.frombuffer(block, dtype=np.uint8)
    return np.bitwise_xor(cipher_npa, block_npa).tobytes()

In [19]:
ciphertext = xor_encrypt(binascii.unhexlify(cipher), binascii.unhexlify(xor_block))
print(f"ciphertext, hex: {binascii.hexlify(ciphertext).decode()}")
print(f"ciphertext, bytes: {ciphertext}")

ciphertext, hex: 746865206b696420646f6e277420706c6179
ciphertext, bytes: b"the kid don't play"


# Single-byte XOR cipher<a name="prob3"></a>

In [21]:
ciphertext = "1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736"

Iterate over all possible XOR values from 0x00 to 0xFF, and then compare all characters in the candidate cipher output against valid printable ASCII values, which range from 0x20 - 0x7E.

**Assumption** is that the output is printable ASCII.

In [22]:
ciphertext_b = binascii.unhexlify(ciphertext)
possible_results = {}
for i in range(0x100):
    block = np.ones(len(ciphertext_b), dtype=np.uint8) * i
    candidate = xor_encrypt(ciphertext_b, block.tobytes())
    if True not in map(lambda x: x < 0x20 or x > 0x7f, candidate):
        if True in map(lambda word: word in words.words(), candidate.decode().split()):
            possible_results[i] = candidate.decode()
for k, v in possible_results.items():
    print(f"XOR: 0x{k:02x}; cipher: {v}")

XOR: 0x58; cipher: Cooking MC's like a pound of bacon


# Detect single-character XOR<a name="prob4"></a>

In [3]:
class CipherCandidate:
    row: int
    xor: int
    cipher: str
    ascii_score: float
    def __init__(self, row, xor, cipher, ascii_score):
        self.row = row
        self.xor = xor
        self.cipher = cipher
        self.ascii_score = ascii_score
    def __str__(self):
        return f"Row: {self.row}; XOR: 0x{self.xor:02x}; Cipher: {self.cipher}; ASCII score: {self.ascii_score}"

In [4]:
def isprintableascii(x):
    if x >= 0x9 and x <= 0x7e:
        return 1
    else:
        return 0

def ascii_score(cipher: bytes) -> float:
    isascii = np.array([x for x in map(isprintableascii, cipher)])
    return np.sum(isascii) / len(cipher)

In [28]:
results = []
ascii_threshold = 0.9
fd = open("set1/4.txt")
line = fd.readline()
idx = 0
while line:
    # remove newline
    if line.endswith("\n"):
        line = line[:-1]
    line_b = binascii.unhexlify(line)
    for xor in range(0x100):
        xor_block = np.ones(len(line_b), dtype=np.uint8) * xor
        cipher = xor_encrypt(line_b, xor_block.tobytes())
        if True not in map(lambda x: x < 0x9 or x > 0x7e, cipher):
            score = ascii_score(cipher)
            if score > ascii_threshold:
                results.append(CipherCandidate(idx, xor, cipher, score))
    line = fd.readline()
    idx += 1
for r in results:
    if True in map(lambda word: word in words.words(), r.cipher.decode().split()):
        print(r)
fd.close()

Row: 149; XOR: 0x4a; Cipher: b"p\t\x19bk^\x13qt|\x0e'f\x14T\x12dy\x19}HczyUk\\N\x1d "; ASCII score: 1.0
Row: 149; XOR: 0x4e; Cipher: b't\r\x1dfoZ\x17upx\n#b\x10P\x16`}\x1dyLg~}QoXJ\x19$'; ASCII score: 1.0
Row: 149; XOR: 0x63; Cipher: b"Y 0KBw:X]U'\x0eO=};MP0TaJSP|Bug4\t"; ASCII score: 1.0
Row: 165; XOR: 0x44; Cipher: b'hS\x1eU\x11y\x0fORF\x1aap\\M `\x1fV\x1a\x19*\x1d\x19YnCT\x1cO'; ASCII score: 1.0
Row: 165; XOR: 0x45; Cipher: b'iR\x1fT\x10x\x0eNSG\x1b`q]L!a\x1eW\x1b\x18+\x1c\x18XoBU\x1dN'; ASCII score: 1.0
Row: 165; XOR: 0x46; Cipher: b'jQ\x1cW\x13{\rMPD\x18cr^O"b\x1dT\x18\x1b(\x1f\x1b[lAV\x1eM'; ASCII score: 1.0
Row: 165; XOR: 0x47; Cipher: b'kP\x1dV\x12z\x0cLQE\x19bs_N#c\x1cU\x19\x1a)\x1e\x1aZm@W\x1fL'; ASCII score: 1.0
Row: 170; XOR: 0x35; Cipher: b'Now that the party is jumping\n'; ASCII score: 1.0
Row: 170; XOR: 0x61; Cipher: b"\x1a;#t <5 t <1t$5& -t='t>!9$=:3^"; ASCII score: 1.0
Row: 170; XOR: 0x72; Cipher: b'\t(0g3/&3g3/"g7&53>g.4g-2*7.) M'; ASCII score: 1.0
Row: 195; XOR: 0x22

Interesting to see how even in wrong XOR of data sometimes valid words fall out. 

In any case, the correct answer is row 170, with a repeating XOR of 0x35.

# Implementing repeating-key XOR<a name="prob5"></a>

In [25]:
phrase = "Burning 'em, if you ain't quick and nimble\nI go crazy when I hear a cymbal"

In [26]:
xor_block = b"ICE"

In [73]:
def xor_block_encrypt(cipher: bytes, xor_block: bytes) -> bytes:
    while len(cipher) > 0:
        if len(xor_block) > len(cipher):
            xor_block = xor_block[:len(cipher)]
            yield xor_encrypt(cipher, xor_block)
        segment = cipher[:len(xor_block)]
        cipher = cipher[len(xor_block):]
        yield xor_encrypt(segment, xor_block)

In [28]:
phrase_b = bytes(phrase.encode("utf8"))
result = b""
for ciphertext in xor_block_encrypt(phrase_b, xor_block):
    result += ciphertext
print(binascii.hexlify(result).decode())

0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f282f


# Breaking repeating-key XOR<a name="prob6"></a>

## Implement Hamming Distance function
It's worth noting that the XOR operation itself will naturally generate a binary stream of where the two strings are different. Counting up the number of 1's in the output binary stream will result in the Hamming Distance between the two strings.

In [30]:
def hamming(str1: bytes, str2: bytes) -> int:
    diff = xor_encrypt(str1, str2)
    dist = 0
    for n in diff:
        a = np.array([i for i in map(lambda x: int(x), list(f"{n:08b}"))])
        dist += np.sum(a)
    return dist

Make sure the hamming distance between strings "`this is a test`" and "`wokka wokka!!!`" is `37`.

In [30]:
str1 = b"this is a test"
str2 = b"wokka wokka!!!"
hamming(str1, str2)

37

## Determine XOR keysize

In [37]:
from collections import namedtuple
XORKeysize = namedtuple("XORKeysize", ["keysize", "hamming_dist", "hamming_dist_normal"])

In [38]:
import base64
with open("set1/6.txt") as fd:
    data = fd.read()
data = data.replace("\n", "")
data = base64.b64decode(data)

In [44]:
keysize_tests = []
for i in range(2, 41):
    h = hamming(data[:i], data[i:2*i])
    h1 = hamming(data[:i], data[2*i:3*i])
    h2 = hamming(data[i:2*i], data[2*i:3*i])
    h3 = hamming(data[2*i:3*i], data[3*i:4*i])
    h4 = hamming(data[:i], data[3*i:4*i])
    h5 = hamming(data[i:2*i], data[3*i:4*i])
    h_avg = (h + h1 + h2 + h3 + h4 + h5) / 6
    keysize_tests.append(XORKeysize(i, h_avg, h_avg/i))
# pprint.pprint(keysize_tests)
keysize_tests.sort(key=lambda x: x.hamming_dist_normal)
pprint.pprint(keysize_tests)

[XORKeysize(keysize=29, hamming_dist=79.66666666666667, hamming_dist_normal=2.7471264367816093),
 XORKeysize(keysize=5, hamming_dist=14.5, hamming_dist_normal=2.9),
 XORKeysize(keysize=2, hamming_dist=6.0, hamming_dist_normal=3.0),
 XORKeysize(keysize=24, hamming_dist=72.5, hamming_dist_normal=3.0208333333333335),
 XORKeysize(keysize=7, hamming_dist=21.5, hamming_dist_normal=3.0714285714285716),
 XORKeysize(keysize=6, hamming_dist=18.5, hamming_dist_normal=3.0833333333333335),
 XORKeysize(keysize=19, hamming_dist=58.833333333333336, hamming_dist_normal=3.0964912280701755),
 XORKeysize(keysize=20, hamming_dist=62.0, hamming_dist_normal=3.1),
 XORKeysize(keysize=3, hamming_dist=9.333333333333334, hamming_dist_normal=3.111111111111111),
 XORKeysize(keysize=8, hamming_dist=25.0, hamming_dist_normal=3.125),
 XORKeysize(keysize=28, hamming_dist=88.33333333333333, hamming_dist_normal=3.1547619047619047),
 XORKeysize(keysize=30, hamming_dist=95.33333333333333, hamming_dist_normal=3.17777777777

It seems that the smallest normalized hamming distance calculated is when the keysize is 29 bytes

In [45]:
xor_keysize = 29

## Determine XOR key

We're going to trim the data so that it is evenly divisble by the keysize, and then convert it into a matrix.

In [46]:
trimsize = len(data) % xor_keysize
if trimsize > 0:
    data = data[:(-1 * trimsize)]
data_block = []
for i in range(0, len(data), xor_keysize):
    data_block.append(list(data[i:i+xor_keysize]))
data_block = np.array(data_block, dtype=np.uint8)

We're going to measure the mean square error (MSE) of the letter histogram of each row against a histogram of letter frequency of every letter in the English language.

$MSE=\frac{1}{n}\sum_{i=0}^n (Y-\hat{Y})^2$

In [9]:
with open("english_letter_frequency.json") as fd:
    letter_freq = json.load(fd)

def mse(y: np.ndarray, y_hat: np.ndarray):
    return ((y - y_hat) ** 2).mean()

def ascii_score(cipher: bytes) -> float:
    alphabet = b"abcdefghijklmnopqrstuvwxyz-"
    letter_count = {}
    for letter in alphabet:
        letter_count[letter] = 1
    
    for letter in cipher:
        if letter in alphabet:
            letter_count[letter] += 1
            
    letter_freq_a = np.array(list(letter_freq.values()))
    letter_count_a = np.array(list(letter_count_a.values()))
    return mse(letter_freq_a, letter_count_a)

In [51]:
XORCandidate = namedtuple("XORCandidate", ["byte_position", "byte_value", "ascii_score"])

ascii_threshold = 0.9

# TODO: implement a more sophisticated ASCII character scoring technique

def isprintableascii(x):
    if x >= 0x9 and x <= 0x7e:
        return 1
    else:
        return 0

candidates = {}
for pos in range(xor_keysize):
    xor_candidates = []
    ciphertext = data_block[:, pos]
    for xor in range(0x100):
        block = np.ones(len(ciphertext), dtype=np.uint8) * xor
        cipher = np.bitwise_xor(ciphertext, block).tobytes()
        isascii = np.array([x for x in map(isprintableascii, cipher)])
        ascii_score = np.sum(isascii) / len(ciphertext)
        if ascii_score >= ascii_threshold:
            xor_candidates.append(XORCandidate(pos, xor, ascii_score))
    candidates[pos] = xor_candidates
            
# pprint.pprint(candidates)

## Sort by top ASCII scores
The process has returned multiple possible XOR values. The likely reason for this is that multiple XOR masked values resulted in still-valid ASCII values. Clean up `candidates` by keeping top ASCII score.

In [52]:
for byte_pos, xor_candidates in candidates.items():
    print(f"Byte Position: {byte_pos}")
    max_ascii_score = 0
    for candidate in xor_candidates:
        if candidate.ascii_score > max_ascii_score:
            max_ascii_score = candidate.ascii_score
    for candidate in xor_candidates:
        if candidate.ascii_score >= max_ascii_score:
            print(candidate)

Byte Position: 0
XORCandidate(byte_position=0, byte_value=0, ascii_score=1.0)
XORCandidate(byte_position=0, byte_value=1, ascii_score=1.0)
XORCandidate(byte_position=0, byte_value=2, ascii_score=1.0)
XORCandidate(byte_position=0, byte_value=3, ascii_score=1.0)
XORCandidate(byte_position=0, byte_value=4, ascii_score=1.0)
XORCandidate(byte_position=0, byte_value=6, ascii_score=1.0)
XORCandidate(byte_position=0, byte_value=16, ascii_score=1.0)
XORCandidate(byte_position=0, byte_value=17, ascii_score=1.0)
XORCandidate(byte_position=0, byte_value=18, ascii_score=1.0)
XORCandidate(byte_position=0, byte_value=19, ascii_score=1.0)
XORCandidate(byte_position=0, byte_value=20, ascii_score=1.0)
XORCandidate(byte_position=0, byte_value=22, ascii_score=1.0)
XORCandidate(byte_position=0, byte_value=23, ascii_score=1.0)
XORCandidate(byte_position=0, byte_value=64, ascii_score=1.0)
XORCandidate(byte_position=0, byte_value=65, ascii_score=1.0)
XORCandidate(byte_position=0, byte_value=75, ascii_score=1.

In [53]:
# xor_block = bytes([107, 109, 97, 109, 107])
# result = b""
# for ciphertext in xor_block_encrypt(data, xor_block):
#     result += ciphertext
# result.decode()

# AES in ECB mode<a name="prob7"></a>

In [9]:
key = b"YELLOW SUBMARINE"
decipher = AES.new(key, AES.MODE_ECB)

with open("set1/7.txt") as fd:
    data = fd.read()
data = data.replace("\n", "")
data = base64.b64decode(data)

ciphertext = decipher.decrypt(data).decode()
pprint.pprint(ciphertext)

("I'm back and I'm ringin' the bell \n"
 "A rockin' on the mike while the fly girls yell \n"
 'In ecstasy in the back of me \n'
 "Well that's my DJ Deshay cuttin' all them Z's \n"
 "Hittin' hard and the girlies goin' crazy \n"
 "Vanilla's on the mike, man I'm not lazy. \n"
 '\n'
 "I'm lettin' my drug kick in \n"
 'It controls my mouth and I begin \n'
 'To just let it flow, let my concepts go \n'
 "My posse's to the side yellin', Go Vanilla Go! \n"
 '\n'
 "Smooth 'cause that's the way I will be \n"
 "And if you don't give a damn, then \n"
 "Why you starin' at me \n"
 "So get off 'cause I control the stage \n"
 "There's no dissin' allowed \n"
 "I'm in my own phase \n"
 'The girlies sa y they love me and that is ok \n'
 "And I can dance better than any kid n' play \n"
 '\n'
 "Stage 2 -- Yea the one ya' wanna listen to \n"
 "It's off my head so let the beat play through \n"
 'So I can funk it up and make it sound good \n'
 '1-2-3 Yo -- Knock on some wood \n'
 'For good luck, I like my rhym

# Detect AES in ECB mode<a name="prob8"></a>

In [5]:
import pyplot as plt

Because ECB is stateless and deterministic, every N-byte block in each ciphertext is going to be searched in itself.

In [26]:
def exists_repeated_bytes(data, num_of_bytes) -> bytes:
    for idx in range(len(data) - num_of_bytes):
        segment = data[idx: idx+num_of_bytes]
        if data[idx+1:].find(segment) > -1:
            return segment
    return None

In [37]:
with open("set1/8.txt") as fd:
    data = fd.read()
    
data = data.split("\n")
data = data[:-1] # remove the last entry, because of newline

for idx, line in enumerate(data):
    repeated_bytes = exists_repeated_bytes(binascii.unhexlify(line), 16)
    if repeated_bytes is not None:
        print(f"Ciphertext {idx} has repeating bytes {binascii.hexlify(repeated_bytes).decode()}")
        print(data[idx])
    

Ciphertext 132 has repeating bytes 08649af70dc06f4fd5d2d69c744cd283
d880619740a8a19b7840a8a31c810a3d08649af70dc06f4fd5d2d69c744cd283e2dd052f6b641dbf9d11b0348542bb5708649af70dc06f4fd5d2d69c744cd2839475c9dfdbc1d46597949d9c7e82bf5a08649af70dc06f4fd5d2d69c744cd28397a93eab8d6aecd566489154789a6b0308649af70dc06f4fd5d2d69c744cd283d403180c98c8f6db1f2a3f9c4040deb0ab51b29933f2c123c58386b06fba186a
